# 장바구니 분석
장바구니 분석은 고객의 구매 데이터를 분석하여 어떤 상품들이 함께 구매되는 경향이 있는지 파악하는 기법입니다. 즉, "A 상품을 구매한 고객은 B 상품도 함께 구매할 가능성이 높다"와 같은 형태의 연관 규칙을 찾아내는 것이죠.

In [1]:
# mlxtend 라이브러리가 없다면 설치합니다.
# pip install mlxtend

import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [4]:
dataset = [['우유', '계란', '빵'],
           ['기저귀', '맥주', '땅콩'],
           ['우유', '기저귀', '맥주', '콜라'],
           ['빵', '버터', '잼'],
           ['기저귀', '맥주', '물티슈'],
           ['우유', '빵'],
           ['기저귀', '맥주', '과자'],
           ['계란', '빵', '버터', '잼']]

te = TransactionEncoder()
te_array = te.fit_transform(dataset)         # 원핫인코딩 - 있으면 true, 없으면 false

df = pd.DataFrame(te_array, columns=te.columns_)
df

계란     과자    기저귀     땅콩     맥주    물티슈     버터      빵     우유      잼     콜라
0   True  False  False  False  False  False  False   True   True  False  False
1  False  False   True   True   True  False  False  False  False  False  False
2  False  False   True  False   True  False  False  False   True  False   True
3  False  False  False  False  False  False   True   True  False   True  False
4  False  False   True  False   True   True  False  False  False  False  False
5  False  False  False  False  False  False  False   True   True  False  False
6  False   True   True  False   True  False  False  False  False  False  False
7   True  False  False  False  False  False   True   True  False   True  False

## 장바구니 분석의 핵심 지표 📊
연관 규칙의 유용성을 판단하기 위해 주로 사용되는 세 가지 핵심 지표가 있습니다.

* 지지도 (Support): 전체 거래 중에서 특정 상품(조합)이 포함된 거래의 비율입니다.

1. Support(A): A를 포함한 거래 수 / 전체 거래 수
2. Support(A→B): A와 B를 모두 포함한 거래 수 / 전체 거래 수

In [5]:
frequent_item = apriori(df, min_support=0.3, use_colnames=True)
frequent_item

support   itemsets
0    0.500      (기저귀)
1    0.500       (맥주)
2    0.500        (빵)
3    0.375       (우유)
4    0.500  (맥주, 기저귀)

* 신뢰도 (Confidence): 특정 상품 A를 구매했을 때, 다른 상품 B도 함께 구매할 조건부 확률입니다.

1. Confidence(A→B): A와 B를 모두 포함한 거래 수 / A를 포함한 거래 수

예: '기저귀'를 구매한 고객이 '맥주'도 함께 구매할 확률. 신뢰도가 높을수록 A 구매 시 B 구매로 이어질 가능성이 높다는 의미입니다.       

* 향상도 (Lift): 상품 A의 구매가 상품 B의 구매 확률에 얼마나 영향을 미치는지를 나타내는 지표입니다.

1. Lift(A→B): Support(A→B) / (Support(A) X Support(B))
2. Lift가 1이면, A와 B가 서로 독립이라는 의미입니다. (A구매가 B구매에 영향이 없음.)
3. Lift가 1보다 크면, A를 구매하면 B를 구매할 확률이 높아집니다. 작다면 반대입니다.

In [8]:
rules = association_rules(frequent_item, min_threshold=0.7)    # 최소 신뢰도 0.7
rules[["antecedents", "consequents", "support", "confidence", "lift"]]

# antecedents : 조건부에 맥주가 있는 경우
# consequents : 결과부에 기저귀가 있는 경우

# support (지지도)      : 전체 거래 중 맥주와 기저귀를 함께 구매한 비율 50%
# confidence (신뢰도)   : 맥주를 구매할 고객이 기저귀를 구매할 확률이 100%
# lift (향상도)         : 맥주를 구매하는 것이 기저귀를 구매할 확률을 2배 높인다.

antecedents consequents  support  confidence  lift
0        (맥주)       (기저귀)      0.5         1.0   2.0
1       (기저귀)        (맥주)      0.5         1.0   2.0

## 연습문제
1. groceries.csv 파일을 활용해서 장바구니 분석을 하세요.
2. '홀밀크'와 '요거트'가 일반적으로 함께 구매될 확률에 비해, '커드'가 구매되었을 때 이 둘을 함께 구매할 확률이 약 몇 배 더 높나요?

In [46]:
import pandas as pd
df = pd.read_csv("Data/groceries.csv")

box = []
for i in range(len(df)):
    row = df.iloc[i].dropna().tolist()[1:]    # 한줄씩꺼내서 list로
    box.append(row)

box

[['citrus fruit', 'semi-finished bread', 'margarine', 'ready soups'],
 ['tropical fruit', 'yogurt', 'coffee'],
 ['whole milk'],
 ['pip fruit', 'yogurt', 'cream cheese', 'meat spreads'],
 ['other vegetables',
  'whole milk',
  'condensed milk',
  'long life bakery product'],
 ['whole milk', 'butter', 'yogurt', 'rice', 'abrasive cleaner'],
 ['rolls/buns'],
 ['other vegetables',
  'UHT-milk',
  'rolls/buns',
  'bottled beer',
  'liquor (appetizer)'],
 ['potted plants'],
 ['whole milk', 'cereals'],
 ['tropical fruit',
  'other vegetables',
  'white bread',
  'bottled water',
  'chocolate'],
 ['citrus fruit',
  'tropical fruit',
  'whole milk',
  'butter',
  'curd',
  'yogurt',
  'flour',
  'bottled water',
  'dishes'],
 ['beef'],
 ['frankfurter', 'rolls/buns', 'soda'],
 ['chicken', 'tropical fruit'],
 ['butter', 'sugar', 'fruit/vegetable juice', 'newspapers'],
 ['fruit/vegetable juice'],
 ['packaged fruit/vegetables'],
 ['chocolate'],
 ['specialty bar'],
 ['other vegetables'],
 ['butter mi

In [47]:
te = TransactionEncoder()
te_array = te.fit_transform(box)         # 원핫인코딩 - 있으면 true, 없으면 false

df = pd.DataFrame(te_array, columns=te.columns_)
df

Instant food products  UHT-milk  abrasive cleaner  artif. sweetener  \
0                     False     False             False             False   
1                     False     False             False             False   
2                     False     False             False             False   
3                     False     False             False             False   
4                     False     False             False             False   
...                     ...       ...               ...               ...   
9830                  False     False             False             False   
9831                  False     False             False             False   
9832                  False     False             False             False   
9833                  False     False             False             False   
9834                  False     False             False             False   

      baby cosmetics  baby food   bags  baking powder  bathroom cleaner  \
0              False      False  False          False             False   
1              False      False  False          False             False   
2              False      False  False          False             False   
3              False      False  False          False             False   
4              False      False  False          False             False   
...              ...        ...    ...            ...               ...   
9830           False      False  False          False             False   
9831           False      False  False          False             False   
9832           False      False  False          False             False   
9833           False      False  False          False             False   
9834           False      False  False          False             False   

       beef  ...  turkey  vinegar  waffles  whipped/sour cream  whisky  \
0     False  ...   False    False    False               False   False   
1     False  ...   False    False    False               False   False   
2     False  ...   False    False    False               False   False   
3     False  ...   False    False    False               False   False   
4     False  ...   False    False    False               False   False   
...     ...  ...     ...      ...      ...                 ...     ...   
9830   True  ...   False    False    False                True   False   
9831  False  ...   False    False    False               False   False   
9832  False  ...   False    False    False               False   False   
9833  False  ...   False    False    False               False   False   
9834  False  ...   False     True    False               False   False   

      white bread  white wine  whole milk  yogurt  zwieback  
0           False       False       False   False     False  
1           False       False       False    True     False  
2           False       False        True   False     False  
3           False       False       False    True     False  
4           False       False        True   False     False  
...           ...         ...         ...     ...       ...  
9830        False       False        True   False     False  
9831        False       False       False   False     False  
9832        False       False       False    True     False  
9833        False       False       False   False     False  
9834        False       False       False   False     False  

[9835 rows x 169 columns]

In [72]:
items = apriori(df, min_support=0.01, use_colnames=True)
items

support                                       itemsets
0    0.033452                                     (UHT-milk)
1    0.017692                                (baking powder)
2    0.052466                                         (beef)
3    0.033249                                      (berries)
4    0.026029                                    (beverages)
..        ...                                            ...
328  0.011998  (tropical fruit, whole milk, root vegetables)
329  0.014540          (whole milk, yogurt, root vegetables)
330  0.010473                     (whole milk, yogurt, soda)
331  0.015150           (tropical fruit, whole milk, yogurt)
332  0.010880       (whole milk, whipped/sour cream, yogurt)

[333 rows x 2 columns]

In [76]:
rules = association_rules(items, min_threshold=0.01)
rules = rules[["antecedents", "consequents", "support", "confidence", "lift"]]

# lift (향상도)로 파악

rules.sort_values("lift", ascending = False)

antecedents                       consequents  \
460                            (curd)              (whole milk, yogurt)   
457              (whole milk, yogurt)                            (curd)   
438  (other vegetables, citrus fruit)                 (root vegetables)   
443                 (root vegetables)  (other vegetables, citrus fruit)   
559        (other vegetables, yogurt)              (whipped/sour cream)   
..                                ...                               ...   
405                            (soda)                      (whole milk)   
120                            (soda)                    (citrus fruit)   
121                    (citrus fruit)                            (soda)   
87                       (rolls/buns)                     (canned beer)   
86                      (canned beer)                      (rolls/buns)   

      support  confidence      lift  
460  0.010066    0.188931  3.372304  
457  0.010066    0.179673  3.372304  
438  0.010371    0.359155  3.295045  
443  0.010371    0.095149  3.295045  
559  0.010168    0.234192  3.267062  
..        ...         ...       ...  
405  0.040061    0.229738  0.899112  
120  0.012811    0.073469  0.887680  
121  0.012811    0.154791  0.887680  
87   0.011286    0.061360  0.789888  
86   0.011286    0.145288  0.789888  

[618 rows x 5 columns]

### 홀밀크와 요거드틀 구매하는 것이  커드를 구매할 확률을 3.37배 높인다.